# Iris project

[The Iris Dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) can be found in the scikit-learn library. See more information in the official documentation. It consitss of 3 different types of irises (targer variables to classify): Setosa, Versicolour and Virginica, but for simplicity just the data for the first 2 are used. The characteristics (Inputs of the model) are: Sepal Length, Sepal Width, Petal Length and Petal Width.
The model is created just using base python and numpy. sklearn is used to download the dataset and to divide it in training and test data. The goal is to create a deep learning model of a neural network using the most basic tools of python without any platform as tensorflow or pytorch to understand deeply all the parts in the architecture.

In [ ]:
# Load the dataset
from sklearn import datasets
from sklearn.model_selection import train_test_split
iris = datasets.load_iris()
data = iris.data 
data.shape
data_target = iris.target
# To simplify the problem, we just use 2 types of irises
data=data[:100]
data_target=data_target[:100]
# Separating the data in training and test
X_train, X_test, y_train, y_test = train_test_split(data, data_target, test_size=0.2, random_state=42)

In [ ]:
import time
import sys
import numpy as np

# Encapsulate our neural network in a class
class IrisNetwork:
    def __init__(self, X_train, y_train, hidden_nodes = 10, learning_rate = 0.1):

        # Assign a seed to our random number generator to ensure we get
        # reproducable results during development 
        np.random.seed(1)
        
        # Build the network to have the number of hidden nodes and the learning rate that
        # were passed into this initializer. Make the same number of input nodes as
        # there are input data points and create a single output node.
        self.init_network(X_train.shape[1], hidden_nodes, 1, learning_rate)         
        
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Store the number of nodes in input, hidden, and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Store the learning rate
        self.learning_rate = learning_rate

        # Initialize weights
        
        # Initialize self.weights_0_1 as a matrix of zeros. These are the weights between
        # the input layer and the hidden layer.
        self.weights_0_1 = np.zeros((X_train.shape[1], self.hidden_nodes))
        
        # Initialize self.weights_1_2 as a matrix of random values. 
        # These are the weights between the hidden layer and the output layer.
        self.weights_1_2 = np.random.normal(0.0, self.hidden_nodes**-0.5, (self.hidden_nodes,self.output_nodes))
        
        # Create the input layer, a two-dimensional matrix with shape 
        # 1 x input_nodes, with all values initialized to zero
        self.layer_0 = np.zeros((1, self.input_nodes))          
                      
    def sigmoid(self,x):
        # Return the result of calculating the sigmoid activation function
        # shown in the lectures
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_output_2_derivative(self,output):
        # Return the derivative of the sigmoid activation function, 
        # where "output" is the original output from the sigmoid fucntion 
        return output*(1-output)

    def train(self, X_train, y_train):
        
        # make sure out we have a matching number of reviews and labels
        assert(len(X_train) == len(y_train))
               
        # Remember when we started for printing time statistics
        start = time.time()
        
        # Keep track of correct predictions to display accuracy during training 
        correct_so_far = 0
        # loop through all the given reviews and run a forward and backward pass,
        # updating weights for every item
        for i in range(len(X_train)):                       

            # Get the next instance and correct measure
            review=X_train[i]
            label=y_train[i]
            self.layer_0=review.reshape((1,len(review)))
            # Implement the forward pass through the network. 
            # That means use the given review to update the input layer, 
            # then calculate values for the hidden layer,
            # and finally calculate the output layer.
            # 
            # Do not use an activation function for the hidden layer,
            # but use the sigmoid activation function for the output layer.
            layer_1=np.dot(self.layer_0, self.weights_0_1)
            layer_2=np.dot(layer_1, self.weights_1_2)
            output_2=self.sigmoid(layer_2)
            # Implement the back propagation pass here. 
            # That means calculate the error for the forward pass's prediction
            # and update the weights in the network according to their
            # contributions toward the error, as calculated via the
            # gradient descent and back propagation algorithms you 
            # learned in class.
            error=label-output_2
            output_error_term=self.sigmoid_output_2_derivative(output_2)*error
            hidden_error=output_error_term*(self.weights_1_2).T
            delta_weights_0_1=np.dot((self.layer_0).T, hidden_error)
            delta_weights_1_2=np.dot((layer_1).T,error)
            self.weights_1_2 += delta_weights_1_2*self.learning_rate
            self.weights_0_1 += delta_weights_0_1*self.learning_rate 
            # Keep track of correct predictions. To determine if the prediction was
            # correct, check that the absolute value of the output error 
            # is less than 0.5. If so, add one to the correct_so_far count.
            if abs(error)<0.5:
                correct_so_far+=1
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the training process. 
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(X_train)))[:4] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4])
            if(i % 25 == 0):
                print("")
            
    def run(self, review):
        # TODO: Run a forward pass through the network, like you did in the
        #       "train" function. That means use the given review to 
        #       update the input layer, then calculate values for the hidden layer,
        #       and finally calculate the output layer.
        self.layer_0=review.reshape((1,len(review)))
        layer_1=np.dot(self.layer_0, self.weights_0_1)
        layer_2=np.dot(layer_1, self.weights_1_2)
        output_2=self.sigmoid(layer_2)
        # The output layer should now contain a prediction. 
        # Return 1 for predictions greater-than-or-equal-to `0.5`, 
        # and 0 otherwise.
        
        if output_2>= 0.5:
            return 1
        else:
            return 0        
        
    def test(self, testing_reviews, testing_labels):
        """
        Attempts to predict the labels for the given testing_reviews,
        and uses the test_labels to calculate the accuracy of those predictions.
        """
        
        # keep track of how many correct predictions we make
        correct = 0

        # we'll time how many predictions per second we make
        start = time.time()

        # Loop through each of the given reviews and call run to predict
        # its label. 
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
           
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")                

In [ ]:
# Starting the model
mlp = IrisNetwork(X_train, y_train, learning_rate=0.1)

In [ ]:
# Test without training
mlp.test(X_test, y_test)

In [ ]:
mlp.train(X_train,y_train)

In [ ]:
# Test after training
mlp.test(X_test, y_test)